# Chapter 5. Object References, Mutability, and Recycling
---

## ToC

1. [del and Garbage Collection](#del-and-garbage-collection)
---

## del and Garbage Collection

The first strange fact about `del` is that it’s not a function, it’s a statement. We write
`del x` and not `del(x)`—although the latter also works, but only because the expressions
`x` and `(x)` usually mean the same thing in Python.

The second surprising fact is that `del` deletes references, not objects. Python’s
garbage collector may discard an object from memory as an indirect result of del, if
the deleted variable was the last reference to the object. Rebinding a variable may also
cause the number of references to an object to reach zero, causing its destruction.

In [17]:
a = [1, 2]
b = a
del a 
b

[1, 2]

Rebinding b to a different object removes the last remaining reference to `[1, 2]`.
Now the garbage collector can discard that object:

In [18]:
b = [3]

![Figure 95](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/95.PNG)

In CPython, the primary algorithm for garbage collection is reference counting. Essentially, each object keeps count of how many references point to it. As soon as that refcount reaches zero, the object is immediately destroyed: CPython calls the `__del__` method on the object (if defined) and then frees the memory allocated to the object. In CPython 2.0, a generational garbage collection algorithm was added to
detect groups of objects involved in reference cycles. Other implementations of Python have more sophisticated
garbage collectors that do not rely on reference counting, which means the __del__
method may not be called immediately when there are no more references to the
object. See [“PyPy, Garbage Collection, and a Deadlock”](https://fpy.li/6-7).

To demonstrate the end of an object’s life, `weakref.finalize` register a callback function to be called when an object is destroyed. See example:

In [21]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('...like tears in the rain.')

ender = weakref.finalize(s1, bye)
ender.alive

True

In [22]:
del s1
ender.alive

True

In [23]:
s2 = 'spam'

...like tears in the rain.


In [24]:
ender.alive

False

The previous example demonstrates that `del` does not delete objects, but
objects may be deleted as a consequence of being unreachable after `del` is used.

### Tricks Python Plays with Immutables

I was surprised to learn that, for a tuple `t`, `t[:]` does not make a copy, but returns a
reference to the same object. You also get a reference to the same tuple if you write `tuple(t)`.

In [25]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [26]:
t3 = t1[:]
t3 is t1

True

The same behavior can be observed with instances of `str`, `bytes`, and `frozenset`.
Note that a frozenset is not a sequence, so `fs[:]` does not work if `fs` is a `frozenset`.
But `fs.copy()` has the same effect: it cheats and returns a reference to the same
object, and not a copy at all!

In [27]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [28]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True

The sharing of string literals is an optimization technique called *interning*.

![Figure 96](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/96.PNG)